In [ ]:
from transformers import AutoTokenizer, LlamaConfig, LlamaTokenizer,LlamaForCausalLM, LlamaTokenizer,TextDataset,DataCollatorForLanguageModeling
from peft import get_peft_config, get_peft_model, PeftModel, PeftConfig ,LoraConfig,TaskType
from transformers import TrainingArguments, Trainer,GenerationConfig,LineByLineTextDataset
from datasets import Dataset , load_dataset
import torch
import textwrap
from torch.nn import CrossEntropyLoss
import numpy as np
import json
import pickle
from transformers import StoppingCriteria, StoppingCriteriaList
import re
from vicuna_setup import vicuna_inference
from vicuna_pubmedqa_eval import model_pubmed_evaluator
import pandas as pd
import csv
from collections import Counter
from evaluation import pub_scores


In [ ]:
#torch.cuda.empty_cache()

In [ ]:
model_path="/home/vicuna-weights-7B"
peft_path="/home/medical-llama/vicuna-lora-7B"
tokenizer = LlamaTokenizer.from_pretrained(model_path)

In [ ]:
#with open("/home/medical-llama/vicuna-lora-7B/adapter_config.json", 'r') as json_file:
#    lora_config = json.load(json_file)

In [ ]:
torch.set_default_tensor_type(torch.cuda.HalfTensor)
model = LlamaForCausalLM.from_pretrained(model_path)
torch.set_default_tensor_type(torch.cuda.FloatTensor)

In [ ]:
#peft_config = PeftConfig.from_pretrained(peft_path)

In [ ]:
model = PeftModel.from_pretrained(
        model,
        peft_path,
        torch_dtype=torch.float16,
)

In [ ]:
model.config.pad_token_id = tokenizer.pad_token_id = 0  # unk
model.config.bos_token_id = 1
model.config.eos_token_id = 2


In [ ]:
# model.eval()

In [ ]:
# model=torch.compile(model)

In [ ]:
pubmed_config=GenerationConfig(
        temperature=0.4,
        top_p=0.75,
        repetition_penalty=1.1,
        beam_size=1,
        max_length=3,
        min_length=1,
        max_time=3,
        max_new_tokens=3, # 3
        forced_bos_token_id=[tokenizer.encode("yes")[1], tokenizer.encode("no")[1], tokenizer.encode("maybe")[1]],
        
    )

In [ ]:
df = pd.read_csv("/home/medical-llama/Pubmedqa/pubmedqa/data/pubmedqa_test.csv")

In [ ]:
def pubmed_template(Question, Context):
        
        prompt_template=f"""
    Respond with "yes","no","maybe" using the context & the question provided.    
     
    Context: Research involving dogs has been instrumental in advancing our understanding of various scientific fields. Dogs are highly trainable and sociable animals.

    Question: Are dogs trainable?

    Answer: yes

    Context: Research involving dogs has been instrumental in advancing our understanding of various scientific fields. Dogs are highly trainable and sociable animals.

    Question: Can dogs be trained to detect and alert individuals to the presence of specific allergens?

    Answer: maybe

    Context: Research involving dogs has been instrumental in advancing our understanding of various scientific fields. Dogs are highly trainable and sociable animals.

    Question: Are Dogs highly introverted & reserved ?

    Answer: no 


    Context:
    {Context}

    Question:
    {Question}

    Answer: 
    
    """
        return prompt_template

In [ ]:
def pubmed_template(Question, Context):
        
        prompt_template=f"""

    your task is to answering research questions relating to medical domain using yes/no/maybe responses. Answer as yes if the context supports the question , answer as no if the context does not support the question , answer as maybe if the context is not clear enough to answer the question.      

    ###
            
    Context: Research involving dogs has been instrumental in advancing our understanding of various scientific fields. Dogs are highly trainable and sociable animals.

    Question: Are dogs trainable?

    Answer: yes

    ###

    Context: Research involving dogs has been instrumental in advancing our understanding of various scientific fields. Dogs are highly trainable and sociable animals.

    Question: Can dogs be trained to detect and alert individuals to the presence of specific allergens?

    Answer: maybe

    ###

    Context: Research involving dogs has been instrumental in advancing our understanding of various scientific fields. Dogs are highly trainable and sociable animals.

    Question: Are Dogs highly introverted & reserved ?

    Answer: no 

    ###

    Context:
    {Context}

    Question:
    {Question}

    Answer: 
    
    """
        return prompt_template

In [ ]:
vicuna_inference(pubmed_template(Question=df['Question'][5],Context=df['Context'][5]),model=model,tokenizer=tokenizer,config=pubmed_config,task='pubmedqa')


In [ ]:
#print(pubmed_template(Question=df['Question'][0],Context=df['Context'][0]))

In [ ]:
config=GenerationConfig(
        temperature=0.4,
        top_p=0.75,
        repetition_penalty=1.1,
        beam_size=1,
        max_length=128,
        min_length=1,
        max_time=10,
        max_new_tokens=128,)


In [ ]:
print(vicuna_inference("can i take paracetamol & drink alcohol ?",model=model,tokenizer=tokenizer,config=config))


In [ ]:
ans_dict={}
for i in range(int(df.shape[0]/10)):
    pmid=str(df['ID'][i])
    ans=vicuna_inference(pubmed_template(Question=df['Question'][i],Context=df['Context'][i]),model=model,tokenizer=tokenizer,config=pubmed_config)
    ans_dict[pmid]=ans

In [ ]:
ans_dict=model_pubmed_evaluator(model=model,tokenizer=tokenizer,config=pubmed_config,id_=0)
